# COVID-19 Overview
> Tracking coronavirus total cases, deaths and new cases by country. Additionally, a detailed view is provided for the US(by state), Europe

- comments: true
- author: Pratap Vardhan
- categories: [overview, interactive]
- image: images/covid-overview.png
- permalink: /covid-overview/
- hide: false

In [1]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/



In [2]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [3]:
#hide

# FETCH
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [4]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]


dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()

In [5]:
#hide
df_table = (pd.DataFrame(dict(Cases=dfc_cases, Deaths=dfc_deaths, PCases=dfp_cases, PDeaths=dfp_deaths))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

Country/Region     Cases  Deaths    PCases  PDeaths  Cases (+)  \
0               US  30393701  550996  30079971   547051     313730   
1           Brazil  12658109  317646  12320169   303462     337940   
2            India  12149335  162468  11846652   160949     302683   
3           France   4646014   95495   4484659    93535     161355   
4           Russia   4486078   96817   4442492    95010      43586   
5   United Kingdom   4355867  126912   4332922   126684      22945   
6            Italy   3561012  108879   3464543   106799      96469   
7           Turkey   3277880   31385   3120013    30619     157867   
8            Spain   3275819   75305   3247738    74420      28081   
9          Germany   2818630   76389   2744608    75669      74022   
10        Colombia   2397731   63255   2359942    62519      37789   
11       Argentina   2332765   55736   2278115    55092      54650   
12          Poland   2288826   52392   2154821    50860     134005   
13          Mexico   2232910  202633   2214542   200211      18368   
14            Iran   1875234   62569   1830823    62142      44411   

    Deaths (+)  Fatality Rate      Continent  
0         3945            1.8  North America  
1        14184            2.5  South America  
2         1519            1.3           Asia  
3         1960            2.1         Europe  
4         1807            2.2         Europe  
5          228            2.9         Europe  
6         2080            3.1         Europe  
7          766            1.0           Asia  
8          885            2.3         Europe  
9          720            2.7         Europe  
10         736            2.6  South America  
11         644            2.4  South America  
12        1532            2.3         Europe  
13        2422            9.1  North America  
14         427            3.3           Asia

In [6]:
#hide
#delete problematic countries from table
df_table = df_table[~df_table['Country/Region'].isin(['Cape Verde', 'Cruise Ship', 'Kosovo'])]

In [7]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Cases (+)', 'Deaths (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu}
summary

{'updated': Timestamp('2021-03-30 00:00:00'),
 'since': Timestamp('2021-03-25 00:00:00'),
 'Cases': 128123698,
 'Deaths': 2801531,
 'Cases (+)': 2717496,
 'Deaths (+)': 47899,
 'China Cases': 101710,
 'China Deaths': 4841,
 'China Cases (+)': 87,
 'China Deaths (+)': 1,
 'US Cases': 30393701,
 'US Deaths': 550996,
 'US Cases (+)': 313730,
 'US Deaths (+)': 3945,
 'EU Cases': 39366930,
 'EU Deaths': 907806,
 'EU Cases (+)': 956124,
 'EU Deaths (+)': 16308}

In [8]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  1/28/20  \
Country/Region                                                                  
Afghanistan           0        0        0        0        0        0        0   
Albania               0        0        0        0        0        0        0   
Algeria               0        0        0        0        0        0        0   
Andorra               0        0        0        0        0        0        0   
Angola                0        0        0        0        0        0        0   

                1/29/20  1/30/20  1/31/20  ...  3/21/21  3/22/21  3/23/21  \
Country/Region                             ...                              
Afghanistan           0        0        0  ...       10       50       24   
Albania               0        0        0  ...      659      344      303   
Algeria               0        0        0  ...       91       98       94   
Andorra               0        0        0  ...       36       28       46   
Angola                0        0        0  ...       37       24       17   

                3/24/21  3/25/21  3/26/21  3/27/21  3/28/21  3/29/21  3/30/21  
Country/Region                                                                 
Afghanistan          15       34       28       36        4       28       62  
Albania             448      472      449      425      493      285      304  
Algeria              89      105      114       93       86      110      115  
Andorra              47       49       45       77       41       38       56  
Angola               62       78       47       70       32       69       50  

[5 rows x 434 columns]

In [9]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table,  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

Visualizations by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Source: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). Link to [notebook](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-overview.ipynb), [orignal interactive](https://gramener.com/enumter/covid19/)